## Connecting to postgreSQL

In [110]:
import os
tmp = os.getcwd()
os.chdir(tmp.split("Coding")[0] + "Coding")

In [111]:
from dotenv import load_dotenv

In [112]:
load_dotenv(".env")

True

In [113]:
import psycopg2
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import sqlalchemy

In [114]:
tmp = os.getcwd()
os.chdir(tmp.split("Coding")[0] + "Coding/jmspack")
from jmspack.internal_utils import (postgresql_data_extraction,
                                    postgresql_table_names_list)

In [115]:
# def postgresql_data_extraction(sql_query: str="SELECT * from suggested_energy_intake"):
#     df = pd.DataFrame()
#     try:
#         conn = psycopg2.connect(host=os.getenv("postgresql_host"),
#                                 database="tracker",
#                                 user="tracker",
#                                 password=os.getenv("postgresql_password"))
#         df = pd.read_sql_query(sql_query, conn)
#         _ = conn.close()
#     except:
#         print("I am unable to connect to the database")
    
#     return df

In [116]:
postgresql_table_names_list()

[('django_session',),
 ('suggested_energy_intake',),
 ('sleep_data',),
 ('django_migrations',),
 ('django_content_type',),
 ('auth_permission',),
 ('auth_group',),
 ('auth_group_permissions',),
 ('auth_user_groups',),
 ('auth_user_user_permissions',),
 ('auth_user',),
 ('django_admin_log',)]

In [117]:
df = postgresql_data_extraction(table_name = 'sleep_data')

In [118]:
df

,sleep_log_entry_id,timestamp,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness


In [119]:
# def aws_data_extraction(filepath: str, bucket="neurocast-vm-shared", **kwargs):
#     s3 = boto3.client(
#             "s3",
#             aws_access_key_id=os.getenv("AWS_KEY"),
#             aws_secret_access_key=os.getenv("AWS_SECRET_KEY"),
#         )
#     obj = s3.get_object(Bucket=bucket, Key=filepath)

#     return pd.read_csv(obj["Body"], **kwargs)

In [120]:
df

,sleep_log_entry_id,timestamp,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness


In [121]:
stop!

SyntaxError: invalid syntax (<ipython-input-121-8c726f94684c>, line 1)

### Making the initial connection to the database

In [122]:
try:
    conn = psycopg2.connect(host="188.166.98.139",
                            database="tracker",
                            user="tracker",
                            password="HAA5oJGgylRToGIrpEKc")
except:
    print("I am unable to connect to the database") 

In [123]:
def delete_postgresql_table(database_name: str,
                             user: str,
                            table_name: str,
                            ):
    try:
        conn = psycopg2.connect(
            host=os.getenv("postgresql_host"),
            database=database_name,
            user=user,
            password=os.getenv("postgresql_password"),
        )
        cursor = conn.cursor()
        _ = cursor.execute("DROP TABLE iris_test;")
        _ = conn.commit()
        _ = conn.close()
    except psycopg2.Error as e:
        print(e)
    
    return f"{table_name} has been deleted from {database_name}"

In [124]:
delete_postgresql_table(database_name="tracker",
                             user="tracker",
                            table_name="iris_test",)

table "iris_test" does not exist



'iris_test has been deleted from tracker'

In [125]:
cursor = conn.cursor()
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cursor.fetchall())
table_list = cursor.fetchall()
list(cursor.fetchall())

[('django_session',), ('suggested_energy_intake',), ('sleep_data',), ('django_migrations',), ('django_content_type',), ('auth_permission',), ('auth_group',), ('auth_group_permissions',), ('auth_user_groups',), ('auth_user_user_permissions',), ('auth_user',), ('django_admin_log',)]


[]

In [126]:
def postgresql_table_names_list(postgresql_conn: psycopg2.connect):
    table_list = False
    try:
        cursor = postgresql_conn.cursor()
        cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
        table_list = cursor.fetchall()
#         print(exists)
        cursor.close()
    except psycopg2.Error as e:
        print(e)
    return table_list

In [127]:
temp = postgresql_table_names_list(postgresql_conn=conn)

In [128]:
temp

[('django_session',),
 ('suggested_energy_intake',),
 ('sleep_data',),
 ('django_migrations',),
 ('django_content_type',),
 ('auth_permission',),
 ('auth_group',),
 ('auth_group_permissions',),
 ('auth_user_groups',),
 ('auth_user_user_permissions',),
 ('auth_user',),
 ('django_admin_log',)]

### Read in data from suggested_energy_intake table from postgreSQL database

In [129]:
df = pd.read_sql_query("SELECT * from suggested_energy_intake", conn)

# Verify that result of SQL query is stored in the dataframe
display(df.head(1))

conn.close()

,nutrient,daily_requirements,units
0,Calorie,1500.0,kcal


In [130]:
df.head()

,nutrient,daily_requirements,units
0,Calorie,1500.0,kcal
1,Total fat,70.0,g
2,Saturated fat,20.0,g
3,Carbohydrate,260.0,g
4,Total sugars,90.0,g


In [131]:
conn.close()

In [132]:
conn

<connection object at 0x7f7efc1cf040; dsn: 'user=tracker password=xxx dbname=tracker host=188.166.98.139', closed: 1>

In [133]:
df_dtypes_dict = df.dtypes.to_dict()

In [134]:
python_to_sql_dtypes_dict = {"object":"text",
"float64":"float",
"float32": "float",
"int8": "int",
"int32": "int", 
 "int64": "int",
"datetime64[ns]": "timestamp"}

In [135]:
columns_dtypes_list = [f"{column} {python_to_sql_dtypes_dict[str(df_dtypes_dict[column])]}" for column in df.columns.tolist()]

In [136]:
table_name = "suggested_energy_intake"
columns_dtypes_str = ',\n'.join(columns_dtypes_list)
print(f"""CREATE TABLE {table_name} (
            {columns_dtypes_str}
)""")

CREATE TABLE suggested_energy_intake (
            nutrient text,
daily_requirements float,
units text
)


In [137]:
def create_table_based_on_df(df: pd.DataFrame,
                             database_name: str,
                             user: str,
                            table_name: str,
                            ):
    python_to_sql_dtypes_dict = {"object":"text",
                                "float64":"float",
                                "float32": "float",
                                "int8": "int",
                                "int32": "int", 
                                 "int64": "int",
                                "datetime64[ns]": "timestamp"}
    df_dtypes_dict = df.dtypes.to_dict()
    columns_dtypes_list = [f"{column} {python_to_sql_dtypes_dict[str(df_dtypes_dict[column])]}" for column in df.columns.tolist()]
    columns_dtypes_str = ',\n'.join(columns_dtypes_list)
    table_string = f"""CREATE TABLE {table_name} (
                {columns_dtypes_str}
    )"""
    
    try:
        conn = psycopg2.connect(
            host=os.getenv("postgresql_host"),
            database=database_name,
            user=user,
            password=os.getenv("postgresql_password"),
        )
        cursor = conn.cursor()
        _ = cursor.execute(table_string)
        _ = conn.commit()
        _ = conn.close()
    except psycopg2.Error as e:
        print(e)
    
    return table_string

In [163]:
import seaborn as sns

df = sns.load_dataset("iris")

In [164]:
os.getcwd()

'/Users/jamestwose/Coding/jmspack'

In [149]:
df = pd.read_csv("../jms_tracker/DS_stuff/MyFitbitData/JamesTwose/Sleep/sleep_score.csv")

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sleep_log_entry_id     61 non-null     int64  
 1   timestamp              61 non-null     object 
 2   overall_score          61 non-null     int64  
 3   composition_score      61 non-null     int64  
 4   revitalization_score   61 non-null     int64  
 5   duration_score         61 non-null     int64  
 6   deep_sleep_in_minutes  61 non-null     int64  
 7   resting_heart_rate     61 non-null     int64  
 8   restlessness           61 non-null     float64
dtypes: float64(1), int64(7), object(1)
memory usage: 4.4+ KB


In [151]:
df.head()

,sleep_log_entry_id,timestamp,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness
0,31329941146,2021-03-15T07:25:30Z,73,17,20,36,58,62,0.095406
1,31311276181,2021-03-14T07:31:30Z,70,15,17,38,74,63,0.119711
2,31298350770,2021-03-13T08:05:30Z,80,18,22,40,107,63,0.083865
3,31285389428,2021-03-12T07:12:30Z,68,16,19,33,54,64,0.105660
4,31270477702,2021-03-11T07:17:30Z,71,15,18,38,88,64,0.074074


In [143]:
df = df.astype(dict(zip(df.columns.tolist(), ["int", "datetime64[ns]"] + np.repeat("int", 6).tolist() + ["float"])))

In [144]:
type(int(df.loc[0, "overall_score"]))

int

In [145]:
type(int(value_tuple[0]))

int

In [146]:
create_table_based_on_df(df=df,
                         database_name="tracker",
                             user="tracker",
                            table_name="sleep_data",
                            )

relation "sleep_data" already exists



'CREATE TABLE sleep_data (\n                sleep_log_entry_id int,\ntimestamp timestamp,\noverall_score int,\ncomposition_score int,\nrevitalization_score int,\nduration_score int,\ndeep_sleep_in_minutes int,\nresting_heart_rate int,\nrestlessness float\n    )'

In [162]:
add_data_to_sql_table(df=df,
                         database_name="tracker",
                             user="tracker",
                            table_name="sleep_data",
                            )

integer out of range



'INSERT INTO sleep_data (sleep_log_entry_id, timestamp, overall_score, composition_score, revitalization_score, duration_score, deep_sleep_in_minutes, resting_heart_rate, restlessness) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [68]:
df.head(1)

,sleep_log_entry_id,timestamp,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness
0,31329941146,2021-03-15 07:25:30,73,17,20,36,58,62,0.095406


In [153]:
table_name="sleep_data"
database_name="tracker"
user="tracker"

In [154]:
python_to_sql_dtypes_dict = {"object":"text",
                            "float64":"float",
                            "float32": "float",
                            "int8": "int",
                            "int32": "int", 
                             "int64": "int",
                            "datetime64[ns]": "timestamp"}
df_dtypes_dict = df.dtypes.to_dict()
columns_dtypes_list = [f"{column} {python_to_sql_dtypes_dict[str(df_dtypes_dict[column])]}" for column in df.columns.tolist()]
columns_dtypes_str = ',\n'.join(columns_dtypes_list)

In [155]:
columns_dtypes_str

'sleep_log_entry_id int,\ntimestamp text,\noverall_score int,\ncomposition_score int,\nrevitalization_score int,\nduration_score int,\ndeep_sleep_in_minutes int,\nresting_heart_rate int,\nrestlessness float'

In [156]:
columns_string

'sleep_log_entry_id, timestamp, overall_score, composition_score, revitalization_score, duration_score, deep_sleep_in_minutes, resting_heart_rate, restlessness'

In [157]:
insert_string

'INSERT INTO suggested_energy_intake (sleep_log_entry_id, timestamp, overall_score, composition_score, revitalization_score, duration_score, deep_sleep_in_minutes, resting_heart_rate, restlessness) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [158]:
columns_string = ', '.join(df.columns.tolist())
value_placeholder_string = ", ".join(["%s" for x in range(0, df.shape[1])])
insert_string = f"""INSERT INTO {table_name} ({columns_string}) VALUES ({value_placeholder_string})"""
try:
    conn = psycopg2.connect(
        host=os.getenv("postgresql_host"),
        database=database_name,
        user=user,
        password=os.getenv("postgresql_password"),
    )
    cursor = conn.cursor()
    for oid in range(0, df.shape[0]):
        value_list = list(df.loc[oid, df.columns[col_number]] for col_number in range(0, df.shape[1]))
        for idx, x in enumerate(value_list):
            if idx == 1:
                value_list[idx] = str(x)
            elif idx == 8:
                value_list[idx] = float(x)
            else:
                value_list[idx] = int(x)
        value_tuple = tuple(value_list)
        _ = cursor.execute(insert_string, value_tuple)

    _ = conn.commit()
    _ = conn.close()

except psycopg2.Error as e:
    print(e)

integer out of range



In [109]:
# for oid in range(0, df.shape[0]):
#     value_list = list(df.loc[oid, df.columns[col_number]] for col_number in range(0, df.shape[1]))
#     for idx, x in enumerate(value_list):
#         if idx == 1:
#             value_list[idx] = str(x)
#         elif idx == 8:
#             value_list[idx] = float(x)
#         else:
#             value_list[idx] = int(x)
#     value_tuple = tuple(value_list)

In [ ]:
# for oid in range(0, df.shape[0]):
#     for col_number in range(0, df.shape[1]):
#         print(df.loc[oid, df.columns[col_number]])

In [92]:
value_list = list(df.loc[oid, df.columns[col_number]] for col_number in range(0, df.shape[1]))

In [159]:
value_list

[31329941146,
 '2021-03-15T07:25:30Z',
 73,
 17,
 20,
 36,
 58,
 62,
 0.0954063604240282]

In [96]:
for idx, x in enumerate(value_list):
    if idx == 1:
        value_list[idx] = str(x)
    else:
        value_list[idx] = float(x)

In [98]:
tuple(value_list)

(31329941146.0,
 '2021-03-15 07:25:30',
 73.0,
 17.0,
 20.0,
 36.0,
 58.0,
 62.0,
 0.0954063604240282)

In [91]:
test_list = [(4, 5), (6, 7), (1, 4), (8, 10)]
  
# printing original list
print("The original list is : " + str(test_list))
  
# Change Datatype of Tuple Values
# Using enumerate() + loop
# converting to string using str()
for idx, (x, y) in enumerate(test_list):
    test_list[idx] = (x, str(y))
  
# printing result 
print("The converted records : " + str(test_list))

The original list is : [(4, 5), (6, 7), (1, 4), (8, 10)]
The converted records : [(4, '5'), (6, '7'), (1, '4'), (8, '10')]


In [64]:
for oid in range(0, df.shape[0]):
    value_tuple = tuple(df.loc[oid, df.columns[col_number]] for col_number in range(0, df.shape[1]))
    print(value_tuple)

(31329941146, Timestamp('2021-03-15 07:25:30'), 73, 17, 20, 36, 58, 62, 0.0954063604240282)
(31311276181, Timestamp('2021-03-14 07:31:30'), 70, 15, 17, 38, 74, 63, 0.1197110423116615)
(31298350770, Timestamp('2021-03-13 08:05:30'), 80, 18, 22, 40, 107, 63, 0.0838650865998176)
(31285389428, Timestamp('2021-03-12 07:12:30'), 68, 16, 19, 33, 54, 64, 0.1056603773584905)
(31270477702, Timestamp('2021-03-11 07:17:30'), 71, 15, 18, 38, 88, 64, 0.074074074074074)
(31256942267, Timestamp('2021-03-10 07:28:30'), 64, 15, 17, 32, 64, 63, 0.0776566757493188)
(31243052379, Timestamp('2021-03-09 07:32:00'), 72, 18, 15, 39, 76, 61, 0.0784313725490196)
(31230306812, Timestamp('2021-03-08 07:27:00'), 75, 16, 21, 38, 75, 63, 0.0765253360910031)
(31223089474, Timestamp('2021-03-07 08:33:00'), 78, 16, 20, 42, 68, 64, 0.08)
(31223089473, Timestamp('2021-03-06 08:23:00'), 78, 19, 21, 38, 67, 65, 0.0773305084745762)
(31223089472, Timestamp('2021-03-05 07:30:00'), 71, 14, 20, 37, 45, 64, 0.0745562130177514)
(3

In [161]:
def add_data_to_sql_table(df: pd.DataFrame,
                             database_name: str,
                             user: str,
                            table_name: str,
                            ):
    
    columns_string = ', '.join(df.columns.tolist())
    value_placeholder_string = ", ".join(["%s" for x in range(0, df.shape[1])])
    insert_string = f"""INSERT INTO {table_name} ({columns_string}) VALUES ({value_placeholder_string})"""
    try:
        conn = psycopg2.connect(
            host=os.getenv("postgresql_host"),
            database=database_name,
            user=user,
            password=os.getenv("postgresql_password"),
        )
        cursor = conn.cursor()
#         for oid in range(0, df.shape[0]):
#             value_tuple = tuple(df.loc[oid, df.columns[col_number]] for col_number in range(0, df.shape[1]))
#             _ = cursor.execute(insert_string, value_tuple)
        for oid in range(0, df.shape[0]):
            value_list = list(df.loc[oid, df.columns[col_number]] for col_number in range(0, df.shape[1]))
            for idx, x in enumerate(value_list):
                if idx == 1:
                    value_list[idx] = str(x)
                elif idx == 8:
                    value_list[idx] = float(x)
                else:
                    value_list[idx] = int(x)
            value_tuple = tuple(value_list)
            _ = cursor.execute(insert_string, value_tuple)

            _ = conn.commit()
            _ = conn.close()
        
    except psycopg2.Error as e:
        print(e)
    
    return insert_string

In [ ]:
columns_string = ', '.join(df.columns.tolist())
value_placeholder_string = ", ".join(["%s" for x in range(0, df.shape[1])])

In [ ]:
insert_string = f"""INSERT INTO {table_name} ({columns_string}) VALUES ({value_placeholder_string})""", 

In [ ]:
c = conn.cursor()
try: 
    for oid in range(0, df.shape[0]):
#         print(oid)
        c.execute("""INSERT INTO suggested_energy_intake (nutrient, daily_requirements, units) VALUES (%s, %s, %s)""", 
        (df.loc[oid, "Nutrient"], df.loc[oid, "Daily requirements"], df.loc[oid, "Units"]))
except:
    print("Unable to upload data to table")
conn.commit() # <--- makes sure the change is shown in the database
conn.close()
c.close()

### Creating a new table in the database

In [ ]:
# c = conn.cursor()
# try:
#     # Create table
#     c.execute("""CREATE TABLE suggested_energy_intake (
#             nutrient text,
#             daily_requirements decimal,
#             units text
#             )""")
# #     cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")
# except:
#     print("I can't drop our test database!")

# conn.commit() # <--- makes sure the change is shown in the database
# conn.close()
# c.close()

### Uploading data to an already created table

In [ ]:
# c = conn.cursor()
# try: 
#     for oid in range(0, df.shape[0]):
# #         print(oid)
#         c.execute("""INSERT INTO suggested_energy_intake (nutrient, daily_requirements, units) VALUES (%s, %s, %s)""", 
#         (df.loc[oid, "Nutrient"], df.loc[oid, "Daily requirements"], df.loc[oid, "Units"]))
# except:
#     print("Unable to upload data to table")
# conn.commit() # <--- makes sure the change is shown in the database
# conn.close()
# c.close()

In [ ]:
stop!

### Local SQLite database connection - read in data

In [ ]:
con = sqlite3.connect("energy_tracker.db")
df = pd.read_sql_query("SELECT * from suggested_energy_intake", con)

# Verify that result of SQL query is stored in the dataframe
display(df.head(1))

con.close()

In [ ]:
df.columns.tolist()